# Wave Equation Example Guide

## Overview
This guide explains how to implement a 1D wave equation solver using PhysicsNeMo. The wave equation is a second-order partial differential equation that describes the propagation of waves, such as sound waves, water waves, or electromagnetic waves.

## Mathematical Formulation
The 1D wave equation is given by:

$$\begin{align*} u_{tt} &= c^2 u_{xx} \\
u(0,t)&=0,\\
u(\pi,t)&=0,\\
u(x,0)&=\sin(x),\\
u_t(x,0)&=\sin(x).
\end{align*}
$$

where:
- u(x,t) is the wave amplitude
- c is the wave speed
- x is the spatial coordinate
- t is time

## Implementation Steps

### 1. Import Required Libraries
```python
import os
import warnings

import numpy as np
from sympy import Symbol, sin

import physicsnemo.sym
from physicsnemo.sym.hydra import to_yaml, to_absolute_path, instantiate_arch, PhysicsNeMoConfig
from physicsnemo.sym.hydra.utils import compose
from physicsnemo.sym.hydra.config import PhysicsNeMoConfig

from physicsnemo.sym.solver import Solver
from physicsnemo.sym.domain import Domain
from physicsnemo.sym.geometry.primitives_1d import Line1D
from physicsnemo.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
)

from physicsnemo.sym.domain.validator import PointwiseValidator
from physicsnemo.sym.key import Key
from physicsnemo.sym.node import Node

from sympy import Symbol, Function, Number
from physicsnemo.sym.eq.pde import PDE
from physicsnemo.sym.utils.io import ValidatorPlotter

import matplotlib.pyplot as plt
plt.rcParams['image.cmap'] = 'jet'
```

### 2. Define the Wave Equation Class
Create a class that inherits from PDE to define the wave equation:
- $x$ and $t$ are symbolic variables representing spatial and temporal coordinates. 
- These variables are bundled into a `input_variables` dictionary. 
- A function $u$ is created, representing a wave function in terms of x and t. 
- The input c is the wave speed coefficient, treated as a function if it's a string, or a number if it's a float or int. 
- A wave equation is formulated and stored in the equations dictionary 
- `diff` is used to compute the derivates

```python
class WaveEquation1D(PDE):
    name = "WaveEquation1D"

    def __init__(self, c=1.0):
        # Define coordinates
        x = Symbol("x")
        t = Symbol("t")
        
        # Define input variables
        input_variables = {"x": x, "t": t}
        
        # Define the wave function
        u = Function("u")(*input_variables)
        
        # Set wave speed coefficient
        if type(c) is str:
            c = Function(c)(*input_variables)
        elif type(c) in [float, int]:
            c = Number(c)
        
        # Define the wave equation
        self.equations = {}
        self.equations["wave_equation"] = # wave equation
```

### 3. Set Up the Solver
Create a main function to set up and run the solver:
- Define the geometry, equations, instantiate network architectures, and nodes in PhysicsNeMo
- Define the initial condition constraint
$$u(x,0) = \sin(x), u_t(x,0)= \sin(x)$$
- Define the boundary constraint
$$ u(0,t)=u(\pi, t)=0$$
- Define the PDE constraint
$$ u_{tt} = c^2 u_{xx}$$
and set $c=1.0$
- Add validator with exact solution
$$u(x,t) = \sin(x)(\sin(t)+\cos(t))$$
```python
@physicsnemo.sym.main(config_path="conf", config_name="config_wave")
def run(cfg: PhysicsNeMoConfig) -> None:
    # Initialize wave equation with wave speed
    c = # c value
    we = WaveEquation1D(c=c)
    
    # Create neural network
    wave_net = instantiate_arch(
        input_keys=[Key("x"), Key("t")],
        output_keys=[Key("u")],
        cfg=cfg.arch.fully_connected,
    )
    
    # Create nodes for the solver
    nodes = we.make_nodes() + [wave_net.make_node(name="wave_network")]
    
    # Define geometry and time range
    x, t_symbol = Symbol("x"), Symbol("t")
    L = float(np.pi)
    geo = Line1D(0, L)
    time_range = {t_symbol: (0, 2 * L)}
    
    # Create domain
    domain = Domain()
    
    # Add initial condition
    IC = PointwiseInteriorConstraint(
        nodes=nodes,
        geometry=geo,
        outvar={}, #initial condition
        batch_size=cfg.batch_size.IC,
        lambda_weighting={"u": 1.0, "u__t": 1.0},
        parameterization={t_symbol: 0.0},
    )
    domain.add_constraint(IC, "IC")
    
    # Add boundary condition
    BC = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=geo,
        outvar={}, #boundary condition
        lambda_weighting={"u": 1.0},
        batch_size=cfg.batch_size.BC,
        parameterization=time_range,
    )
    domain.add_constraint(BC, "BC")
    
    # Add interior constraint
    interior = PointwiseInteriorConstraint(
        nodes=nodes,
        geometry=geo,
        outvar={}, #PDE constraint
        batch_size=cfg.batch_size.interior,
        parameterization=time_range,
    )
    domain.add_constraint(interior, "interior")
    
    # Add validation data
    deltaT = 0.01
    deltaX = 0.01
    x = np.arange(0, L, deltaX)
    t = np.arange(0, 2 * L, deltaT)
    X, T = np.meshgrid(x, t)
    X = np.expand_dims(X.flatten(), axis=-1)
    T = np.expand_dims(T.flatten(), axis=-1)
    u = # add formula of exact solution
    invar_numpy = {"x": X, "t": T}
    outvar_numpy = {"u": u}
    
    # Create validator
    validator = PointwiseValidator(
        nodes=nodes,
        invar=invar_numpy,
        true_outvar=outvar_numpy,
        batch_size=128,
        plotter=ValidatorPlotter(),
    )
    domain.add_validator(validator)
    
    # Create and run solver
    slv = Solver(cfg, domain)
    slv.solve()
```


## Key Components Explained

1. **Wave Equation Class**:
   - Defines the mathematical formulation of the wave equation
   - Uses SymPy for symbolic computation
   - Handles both constant and variable wave speeds

2. **Neural Network**:
   - Takes x and t as inputs
   - Outputs the wave amplitude u
   - Uses a fully connected architecture

3. **Constraints**:
   - Initial Condition (IC): Sets the initial wave shape and velocity
   - Boundary Condition (BC): Sets the wave amplitude at boundaries
   - Interior: Enforces the wave equation throughout the domain

4. **Validation**:
   - Uses analytical solution for validation
   - Compares neural network predictions with exact solution
   - Visualizes results using matplotlib

## Running the Example

1. Fill in all missing parts
2. Place the configuration file in the conf directory
3. Run the script:
```bash
python wave.py
```

## Expected Results
The solver will:
1. Train the neural network to solve the wave equation
2. Generate validation plots comparing the solution with the analytical result

